# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

In [ ]:
# The dataset includes information on various marketing campaigns conducted by a Portuguese banking institution, with features like age, job, marital status, education, and more. The target variable is whether the client subscribed to a term deposit (yes or no).

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import seaborn as sns
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [4]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [ ]:
# Understand the distribution of features.
# Visualize relationships between features and the target variable.
# Identify any potential outliers or anomalies.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [ ]:
# Preprocess the data
# Encode the categorical variables
categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 
                       'month', 'day_of_week', 'poutcome']

label_encoders = {}
for column in categorical_columns:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])

# Encode the target variable
data['y'] = data['y'].map({'yes': 1, 'no': 0})

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [ ]:
# Split the data into features and target variable
X = data.drop(columns=['y'])
y = data['y']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize the numeric features
numeric_columns = ['age', 'campaign', 'pdays', 'previous', 'emp.var.rate', 
                   'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']

scaler = StandardScaler()
X_train[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])
X_test[numeric_columns] = scaler.transform(X_test[numeric_columns])



### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [ ]:
# Initialize classifiers
knn = KNeighborsClassifier()
log_reg = LogisticRegression(random_state=42, max_iter=1000)
decision_tree = DecisionTreeClassifier(random_state=42)
svm = SVC(probability=True, random_state=42)



### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [ ]:
# Train and evaluate classifiers
classifiers = {
    "K-Nearest Neighbors": knn,
    "Logistic Regression": log_reg,
    "Decision Tree": decision_tree,
    "Support Vector Machine": svm
}

### Problem 9: Score the Model

What is the accuracy of your model?

In [ ]:
# Store the results
results = {}

for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    y_prob = clf.predict_proba(X_test)[:, 1] if hasattr(clf, "predict_proba") else clf.decision_function(X_test)
    
    results[name] = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred),
        "ROC-AUC": roc_auc_score(y_test, y_prob)
    }



### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [ ]:
#  Compare the models using the evaluation metrics.

In [ ]:
# Display the results
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values(by="ROC-AUC", ascending=False)
print(results_df)

In [ ]:
# Visualize the performance metrics
results_df.plot(kind='bar', figsize=(15, 8))
plt.title("Comparison of Classifier Performance")
plt.xlabel("Classifier")
plt.ylabel("Score")
plt.xticks(rotation=45)
plt.legend(loc='lower right')
plt.show()

### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [ ]:
# More feature engineering and exploration
# Determine if we should keep the 'gender' feature
# Check the distribution of 'gender' feature
#gender_counts = data['gender'].value_counts()
#print(gender_counts)

# If the 'gender' feature is imbalanced or does not provide useful information, we can drop it
#if gender_counts['male'] < 0.1 * len(data) or gender_counts['female'] < 0.1 * len(data):
#    data = data.drop(columns=['gender'])

# Hyperparameter tuning and grid search
# For each classifier, perform hyperparameter tuning using grid search

# K-Nearest Neighbors
knn_param_grid = {
    'n_neighbors': [3,  9],
    'weights': ['uniform', 'distance']
}
knn_grid_search = GridSearchCV(knn, knn_param_grid, cv=3,  n_jobs=-1)
knn_grid_search.fit(X_train, y_train)
best_knn = knn_grid_search.best_estimator_
print("KNN Tuning Complete")
# Logistic Regression
log_reg_param_grid = {
    'C': [ 1, 5],
    'solver': ['liblinear', 'lbfgs']
}
log_reg_grid_search = GridSearchCV(log_reg, log_reg_param_grid, cv=3,  n_jobs=-1)
log_reg_grid_search.fit(X_train, y_train)
best_log_reg = log_reg_grid_search.best_estimator_
print("Log Reg Tuning Complete")
# Decision Tree
decision_tree_param_grid = {
    'max_depth': [3,  9],
    'min_samples_split': [2,  4]
}
decision_tree_grid_search = GridSearchCV(decision_tree, decision_tree_param_grid, cv=3 ,  n_jobs=-1)
decision_tree_grid_search.fit(X_train, y_train)
best_decision_tree = decision_tree_grid_search.best_estimator_
print("Decision Tree Tuning Complete")
# Support Vector Machine
svm_param_grid = {
    'C': [ 1, 5],
    'kernel': ['linear', 'rbf']
}
svm_grid_search = HalvingGridSearchCV(svm, svm_param_grid, cv=3, factor=2, n_jobs=-1)
svm_grid_search.fit(X_train, y_train)
best_svm = svm_grid_search.best_estimator_
print("SVD Tuning Complete")
# Print best hyperparameters for each classifier
print("Best Hyperparameters:")
print("K-Nearest Neighbors:", best_knn.get_params())
print("Logistic Regression:", best_log_reg.get_params())
print("Decision Tree:", best_decision_tree.get_params())
print("Support Vector Machine:", best_svm.get_params())
# Adjust performance metric
# Instead of using ROC-AUC, let's use F1-Score as the performance metric
# Fit the models with the best hyperparameters
best_knn.fit(X_train, y_train)
best_log_reg.fit(X_train, y_train)
best_decision_tree.fit(X_train, y_train)
best_svm.fit(X_train, y_train)  # Uncomment this line if you want to fit the SVM model

# Evaluate the models with the adjusted performance metric
results = {}

# K-Nearest Neighbors
y_pred_knn = best_knn.predict(X_test)
results["K-Nearest Neighbors"] = {
    "Accuracy": accuracy_score(y_test, y_pred_knn),
    "Precision": precision_score(y_test, y_pred_knn),
    "Recall": recall_score(y_test, y_pred_knn),
    "F1-Score": f1_score(y_test, y_pred_knn)
}

# Logistic Regression
y_pred_log_reg = best_log_reg.predict(X_test)
results["Logistic Regression"] = {
    "Accuracy": accuracy_score(y_test, y_pred_log_reg),
    "Precision": precision_score(y_test, y_pred_log_reg),
    "Recall": recall_score(y_test, y_pred_log_reg),
    "F1-Score": f1_score(y_test, y_pred_log_reg)
}

# Decision Tree
y_pred_decision_tree = best_decision_tree.predict(X_test)
results["Decision Tree"] = {
    "Accuracy": accuracy_score(y_test, y_pred_decision_tree),
    "Precision": precision_score(y_test, y_pred_decision_tree),
    "Recall": recall_score(y_test, y_pred_decision_tree),
    "F1-Score": f1_score(y_test, y_pred_decision_tree)
}

# Support Vector Machine
y_pred_svm = best_svm.predict(X_test)
results["Support Vector Machine"] = {
    "Accuracy": accuracy_score(y_test, y_pred_svm),
    "Precision": precision_score(y_test, y_pred_svm),
    "Recall": recall_score(y_test, y_pred_svm),
    "F1-Score": f1_score(y_test, y_pred_svm)
}

# Display the results
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values(by="F1-Score", ascending=False)
print("Adjusted Results:")
print(results_df)

# Visualize the performance metrics
results_df.plot(kind='bar', figsize=(15, 8))
plt.title("Comparison of Classifier Performance")
plt.xlabel("Classifier")
plt.ylabel("Score")
plt.xticks(rotation=0)
plt.legend(loc='lower right', prop={'size': 8})  # Set the legend font size to 8
plt.show()

##### Questions